In [ ]:
### Housekeeping ###
import os

### Data ###
import numpy as np
import pandas as pd
import rpy2.robjects as robjects
import scipy.io

### Visualization ###
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

### Machine Learning ###
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
dir = os.path.join(os.path.expanduser('~'), 'Downloads', 'pbl_medip_counts_matrix_blacklist_filtered.RData')
dir

In [ ]:
robjects.r['load'](dir)

In [ ]:
matrix = robjects.r['matri']
matrix

In [ ]:
np_array = np.array(matrix)
np_array

In [ ]:
df = pd.DataFrame(np_array, columns=matrix.colnames).T
df = df[:72]
df

In [ ]:
X = df.values
X

In [ ]:
y = df.index.values
y

In [ ]:
scaled_X = StandardScaler().fit_transform(X)
scaled_X

In [ ]:
pca = PCA(n_components=3)
principal_components = pca.fit_transform(scaled_X)
principal_df = pd.DataFrame(data=principal_components)
principal_df

In [ ]:
final_df = pd.concat([principal_df, pd.DataFrame(y)], axis=1)
final_df.rename(columns={0:'sample'}, inplace=True)
final_df

In [ ]:
fig = px.scatter(final_df,
                    x='PC1', y='PC2',
                    color='sample',
                    width=1000, height=800,
                    opacity=1
                    )

fig.update_traces(marker=dict(size=5,
                              line=dict(width=1,
                                        color='DarkSlateGrey')
                              ),
                  )

fig.show()

In [ ]:
fig = px.scatter_3d(final_df,
                    x='PC1', y='PC2', z='PC3',
                    color='sample',
                    width=1000, height=800,
                    opacity=1
                    )

fig.update_traces(marker=dict(size=5,
                              line=dict(width=1,
                                        color='DarkSlateGrey')
                              ),
                  )

fig.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
final_df.rename_axis(None, inplace=True)
final_df

In [ ]:
final_df.drop(columns=['sample']).values

In [ ]:
inertias = []
ks = range(1,10)

for k in ks:
    km_test = KMeans(n_clusters=k, n_init=10).fit(final_df.drop(columns=['sample']).values)
    inertias.append(km_test.inertia_)

plt.plot(ks, inertias)
plt.xlabel('k cluster number');

In [ ]:
k = 2

kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
kmeans.fit(final_df.drop(columns=['sample']).values)
cluster_assignments = kmeans.labels_
final_df['cluster'] = cluster_assignments
final_df

In [ ]:
plt.scatter(x=final_df['PC1'], y = final_df['PC2'], c=final_df['cluster'])
plt.show()

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.xlabel('Principal Component'); plt.ylabel('% explained variance');

In [ ]:
pca.explained_variance_ratio_ * 100

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.ylim(ymin=0)
plt.title('cumulated share of explained variance')
plt.xlabel('# of principal component used');